In [1]:
import pandas as pd
import numpy as np
#articles.csv available  at https://www.kaggle.com/marlesson/news-of-the-site-folhauol
articles = pd.read_csv('articles.csv')
articles.head()

,title,text,date,category,subcategory,link
0,"Lula diz que está 'lascado', mas que ainda tem...",Com a possibilidade de uma condenação impedir ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
1,"'Decidi ser escrava das mulheres que sofrem', ...","Para Oumou Sangaré, cantora e ativista malines...",2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
2,Três reportagens da Folha ganham Prêmio Petrob...,Três reportagens da Folha foram vencedoras do ...,2017-09-10,poder,NaN,http://www1.folha.uol.com.br/poder/2017/10/192...
3,Filme 'Star Wars: Os Últimos Jedi' ganha trail...,A Disney divulgou na noite desta segunda-feira...,2017-09-10,ilustrada,NaN,http://www1.folha.uol.com.br/ilustrada/2017/10...
4,CBSS inicia acordos com fintechs e quer 30% do...,"O CBSS, banco da holding Elopar dos sócios Bra...",2017-09-10,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/10/1...


In [2]:
articles.shape

(167053, 6)

In [3]:
articles.title[0]

"Lula diz que está 'lascado', mas que ainda tem força como cabo eleitoral"

# Sentencepiece (Words vocab)

In [3]:
#!pip3 install sentencepiece

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.0 MB 494 kB/s eta 0:00:01


In [2]:
import sentencepiece as spm
import re

In [6]:
#titles = articles.title.apply(lambda x: re.sub('[\t]', '', x)).str.cat(sep=' ')
titles = articles.title.apply(lambda x: re.sub('[\t]', '', '<s> '+ x + '</s>')).str.cat(sep=' ')

In [6]:
len(titles)

11725850

In [6]:
titles_file = open('titles.txt', 'w+')
titles_file.write(titles)
titles_file.close()

In [7]:
spm.SentencePieceTrainer.train('--input=titles.txt --model_prefix=m --user_defined_symbols=<s>,</s> --vocab_size=8000')

True

In [3]:
sp = spm.SentencePieceProcessor()
sp.load('m.model')

True

In [4]:
# text => id
print(sp.encode_as_pieces(' <s> Brasil econômia</s>'))
print(sp.encode_as_ids('<s> Brasil econômia</s>'))

# id => text
print(sp.decode_pieces(['▁Brasil', '▁e', 'con', 'ômi', 'a']))
print(sp.decode_ids([34, 6, 714, 5195, 14]))

['▁', '<s>', '▁Brasil', '▁e', 'con', 'ômi', 'a', '</s>']
[3, 1, 37, 7, 685, 4031, 14, 2]
Brasil econômia
se, Russomanno confirmadoa


In [10]:
sp.id_to_piece(1) == '<s>'

True

In [11]:
print(sp.get_piece_size())

# id <=> piece conversion
print(sp.id_to_piece(209))
print(sp.piece_to_id('í'))

print(sp.piece_to_id('__MUST_BE_UNKNOWN__'))

# <unk>, <s>, </s> are defined by default. Their ids are (0, 1, 2)
# <s> and </s> are defined as 'control' symbol.
for id in range(3):
  print(sp.id_to_piece(id), sp.is_control(id))

8000
▁TV
161
0
<unk> False
<s> False
</s> False


# Input Transform

In [8]:
import tensorflow as tf

In [7]:
titles_as_id = []
for title in titles.split(sep='</s>')[:-1]:
    titles_as_id+=sp.encode_as_ids(title)
    titles_as_id.append(sp.piece_to_id('</s>'))
len(titles_as_id)

3074017

In [14]:
sp.decode_ids([8, 1, 132, 23, 20, 146, 18, 1569, 120, 40, 16, 5, 72, 20, 289, 45, 545, 68, 6467, 678, 2])

"a<s> casa ' quein no encerrações sobre paras como que Hillary temceri Figueirense Marta</s>"

In [15]:
sp.id_to_piece(1)

'<s>'

In [16]:
sp.encode_as_ids("<s> Lula diz que está 'lascado', mas que ainda tem força como cabo eleitoral</s>")

[3,
 1,
 160,
 22,
 20,
 158,
 23,
 733,
 63,
 30,
 13,
 6,
 78,
 20,
 319,
 45,
 624,
 72,
 3,
 63,
 200,
 752,
 2]

In [8]:
titles_dataset = tf.data.Dataset.from_tensor_slices(titles_as_id)

In [9]:
seq_len = 25
sequences = titles_dataset.batch(seq_len, drop_remainder=True)

In [10]:
def split_input_target(sentence):
    input_text = sentence[:-1]
    target_text = sentence[1:]
    return input_text, target_text

In [11]:
data_input_target = sequences.map(split_input_target)

In [12]:
for input_text, target_text in data_input_target.take(2):
    print(sp.decode_ids(input_text.numpy().tolist()))
    print(sp.decode_ids(target_text.numpy().tolist()))

<s> Lula diz que está 'lascado', mas que ainda tem força como cabo eleitoral</s> 
<s> Lula diz que está 'lascado', mas que ainda tem força como cabo eleitoral</s> <s>
'Decidi ser escrava das mulheres que sofrem', diz cantora Oumou Sangaré</s>
Decidi ser escrava das mulheres que sofrem', diz cantora Oumou Sangaré</s> 


In [4]:
BATCH_SIZE = 64

BUFFER_SIZE = 1000

batches = data_input_target.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

batches

NameError: name 'data_input_target' is not defined

# Build Model

In [5]:
vocab_size = sp.get_piece_size()

embedding_dim = 256

rnn_units = 1024

In [6]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [16]:
model = build_model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE)

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           2048000   
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 8000)          8200000   
Total params: 14,186,304
Trainable params: 14,186,304
Non-trainable params: 0
_________________________________________________________________


In [18]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [19]:
model.compile(optimizer='adam', loss=loss)

In [9]:
import os
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_latest")
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)

In [23]:
EPOCHS = 3
history = model.fit(batches, epochs=EPOCHS, callbacks=[checkpoint_callback])

Train for 1921 steps
Epoch 1/3
1921/1921 [==============================] - 825s 430ms/step - loss: 2.9921
Epoch 2/3
1921/1921 [==============================] - 826s 430ms/step - loss: 2.8840
Epoch 3/3
1921/1921 [==============================] - 827s 430ms/step - loss: 2.7978


# Restore model

In [10]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (1, None, 256)            2048000   
_________________________________________________________________
gru (GRU)                    (1, None, 1024)           3938304   
_________________________________________________________________
dense (Dense)                (1, None, 8000)           8200000   
Total params: 14,186,304
Trainable params: 14,186,304
Non-trainable params: 0
_________________________________________________________________


# Run Model

In [14]:
def generate_text(model, start_string):
    start_str_ids = sp.encode_as_ids(start_string)
    input_eval = start_str_ids
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated_id = []
    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    temperature = 0.3
    model.reset_states()
    while len(text_generated_id)==0 or (text_generated_id[-1] != 2 and len(text_generated_id)<40):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)
        text_generated_id.append(int(predicted_id))

    #return start_string + ' ' +(sp.decode_ids(text_generated_id))
    return sp.decode_ids(start_str_ids+text_generated_id)

In [19]:
for i in range(0, 10):
    print(generate_text(model, "Em pronunciamento na ONU"))

Em pronunciamento na ONU, Dilma diz estar 'indignado'</s>
Em pronunciamento na ONU, Hollande diz que sanções dos EUA são políticas</s>
Em pronunciamento na ONU, Hollande diz que vai 'seme'</s>
Em pronunciamento na ONU, Dilma defende diplomacia</s>
Em pronunciamento na ONU, Hollande diz que será candidato a presidente do São Paulo</s>
Em pronunciamento na ONU, Hollande diz que sanções foram ameaçados</s>
Em pronunciamento na ONU, Hollande diz que pau de selfie</s>
Em pronunciamento na ONU, Dilma veta correção do 'Sattoday Night Live'</s>
Em pronunciamento na ONU, Dilma veta correção de cálculos de benefícios</s>
Em pronunciamento na ONU, Dilma cita Dilma Rousseff</s>


In [20]:
for i in range(0, 10):
    print(generate_text(model, "Brasil passa"))

Brasil passa a valer pelos EUA</s>
Brasil passa a valer pelo menos até novembro</s>
Brasil passa a valer pela 2a vez seguida</s>
Brasil passa a valer pelo país</s>
Brasil passa EUA por dificuldades financeiras, diz ex-presidente</s>
Brasil passa a valer pela 2a vez seguida</s>
Brasil passa a valer pelo menos R$ 2,7 bi</s>
Brasil passa a valer pelo menos R$ 2,7 bi</s>
Brasil passa a valer pelo menos bloqueio de turistas</s>
Brasil passa a valer entrada de turistas</s>


In [21]:
for i in range(0, 10):
    print(generate_text(model, "Bolsonaro"))

Bolsonaro cobra responsabilidade por 'pau de selfie'</s>
Bolsonaro cobra responsabilidade do governo</s>
Bolsonaro! Nem meríram!</s>
Bolsonaro diz que não é chamado de 'sexual'</s>
Bolsonaro cobra responsabilidade de Dilma</s>
Bolsonaro diz ter 'incisão' reuniões com investidores</s>
Bolsonaro quer tratar de cadernosagem</s>
Bolsonaro escolheu para visitar EUA</s>
Bolsonaro! Nem meu!</s>
Bolsonaro diz ter sido alvo de ataque a jornal</s>


In [22]:
for i in range(0, 10):
    print(generate_text(model, "Lula"))

Lula diz que Dilma terá que 'superar' desoneração da folha</s>
Lula disse que Dilma age para investigações da Lava Jato</s>
Lula diz que Marta terá 'novo normal'</s>
Lula acusa Dilma de 'mentiroso'</s>
Lula nega pedido de liberdade de executivos da empreiteira OAS</s>
Lula acusa Dilma de agente</s>
Lula diz que Dilma não quis ser uma bobagem</s>
Lula afirma que Marta recebeu R$ 500 mi</s>
Lula desabafa contra falta de luz</s>
Lula diz que Dilma terá 'novo normal'</s>


In [18]:
for i in range(0, 10):
    print(generate_text(model, "<s>"))

<s> Com cachê, Inter vence com facilidade e avança à semi do Aberto da Austrália</s>
<s> Antes de ataque, grupo de Charlie Hebdo' chega às bancas em jornal</s>
<s> Leitores comentam aumento salarial de até 12 horas após ônibus</s>
<s>  ⁇ nibus é incendiado na zona leste de São Paulo</s>
<s> O que as crianças estão chegando</s>
<s> Após quase dois anos, Blatter diz que brasileiros estão garantidas na Copa</s>
<s>  ⁇ nibus é incendiado na zona norte de São Paulo</s>
<s>  ⁇ nibus é incendiado na zona leste de São Paulo</s>
<s>  ⁇ ltimas Conversas entre índios e hip-hop</s>
<s> Após temperaturas, nível do Cantareira volta a cair</s>


# Retieving the embeddings

In [24]:
weights = model.layers[0].get_weights()[0]
weights.shape

(8000, 256)

In [25]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for id in range(sp.get_piece_size()):
  vec = weights[id] # skip 0, it's padding.
  out_m.write(sp.id_to_piece(id) + "\n")
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
out_v.close()
out_m.close()